In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Deep_learning_final

/content/drive/MyDrive/Colab Notebooks/Deep_learning_final


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set up GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess the CIFAR-10 dataset
transform = transforms.Compose(
    [
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

trainset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
import torchvision.models as models

def train(model, trainloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")

def evaluate(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# Load pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Modify the first convolutional layer
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

# Remove the first max pooling layer
model.maxpool = nn.Identity()

# Modify the final layer for CIFAR-10
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Train the model
train(model, trainloader, criterion, optimizer, num_epochs=20)

# Evaluate the model
evaluate(model, testloader)

# Save the model"cifar_resnet18_pretrained.pth"
torch.save(model.state_dict(), "lin_cifar_resnet18_pretrained.pth")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 170MB/s]


Epoch [1/20], Loss: 1.0527
Epoch [2/20], Loss: 0.7189
Epoch [3/20], Loss: 0.6249
Epoch [4/20], Loss: 0.5742
Epoch [5/20], Loss: 0.5234
Epoch [6/20], Loss: 0.4977
Epoch [7/20], Loss: 0.4690
Epoch [8/20], Loss: 0.4459
Epoch [9/20], Loss: 0.4284
Epoch [10/20], Loss: 0.4164
Epoch [11/20], Loss: 0.4023
Epoch [12/20], Loss: 0.3865
Epoch [13/20], Loss: 0.3725
Epoch [14/20], Loss: 0.3693
Epoch [15/20], Loss: 0.3595
Epoch [16/20], Loss: 0.3478
Epoch [17/20], Loss: 0.3402
Epoch [18/20], Loss: 0.3382
Epoch [19/20], Loss: 0.3243
Epoch [20/20], Loss: 0.3201
Accuracy: 82.19%


In [15]:
num_classes = 10

# Load DenseNet
densenet = models.densenet121(pretrained=True)
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)
densenet = densenet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_densenet = optim.Adam(densenet.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_densenet, step_size=30, gamma=0.1)
trainloader_densenet = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# Use the same train and evaluate functions as before
train(densenet, trainloader_densenet, criterion, optimizer_densenet, num_epochs=20)
evaluate(densenet, testloader)
torch.save(densenet.state_dict(), "lin_cifar_densenet_pretrained.pth")

Epoch [1/20], Loss: 0.8696
Epoch [2/20], Loss: 0.5532
Epoch [3/20], Loss: 0.4479
Epoch [4/20], Loss: 0.3820
Epoch [5/20], Loss: 0.3243
Epoch [6/20], Loss: 0.2857
Epoch [7/20], Loss: 0.2559
Epoch [8/20], Loss: 0.2231
Epoch [9/20], Loss: 0.2002
Epoch [10/20], Loss: 0.1869
Epoch [11/20], Loss: 0.1692
Epoch [12/20], Loss: 0.1655
Epoch [13/20], Loss: 0.1494
Epoch [14/20], Loss: 0.1439
Epoch [15/20], Loss: 0.1345
Epoch [16/20], Loss: 0.1297
Epoch [17/20], Loss: 0.1298
Epoch [18/20], Loss: 0.1181
Epoch [19/20], Loss: 0.1249
Epoch [20/20], Loss: 0.1149
Accuracy: 83.66%


In [16]:
num_classes = 10

# Load VGG
vgg = models.vgg16(pretrained=True)
vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, num_classes)
vgg = vgg.to(device)

trainloader_VGG = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer_VGG = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_VGG, step_size=30, gamma=0.1)

# Use the same train and evaluate functions as before
train(vgg, trainloader_VGG, criterion, optimizer_VGG, num_epochs=20)
evaluate(vgg, testloader)
torch.save(vgg.state_dict(), "lin_cifar_vgg_pretrained.pth")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 85.1MB/s]


Epoch [1/20], Loss: 0.7392
Epoch [2/20], Loss: 0.4212
Epoch [3/20], Loss: 0.3036
Epoch [4/20], Loss: 0.2192
Epoch [5/20], Loss: 0.1521
Epoch [6/20], Loss: 0.1073
Epoch [7/20], Loss: 0.0836
Epoch [8/20], Loss: 0.0531
Epoch [9/20], Loss: 0.0457
Epoch [10/20], Loss: 0.0360
Epoch [11/20], Loss: 0.0238
Epoch [12/20], Loss: 0.0225
Epoch [13/20], Loss: 0.0215
Epoch [14/20], Loss: 0.0141
Epoch [15/20], Loss: 0.0184
Epoch [16/20], Loss: 0.0217
Epoch [17/20], Loss: 0.0068
Epoch [18/20], Loss: 0.0128
Epoch [19/20], Loss: 0.0076
Epoch [20/20], Loss: 0.0041
Accuracy: 88.70%


In [17]:
from torchvision import models

# Number of classes for CIFAR-10
num_classes = 10

# Define the criterion and scheduler
criterion = nn.CrossEntropyLoss()

# Load Pretrained MobileNetV2
mobilenet = models.mobilenet_v2(pretrained=True)

# Modify the classifier to fit CIFAR-10 (num_classes = 10)
mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, num_classes)
mobilenet = mobilenet.to(device)

# Define the optimizer
optimizer_mobilenet = optim.Adam(mobilenet.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_mobilenet, step_size=30, gamma=0.1)
# Set up the DataLoader (use your existing trainset and testloader)
trainloader_mobilenet = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# Train the model (use your train function)
train(mobilenet, trainloader_mobilenet, criterion, optimizer_mobilenet, num_epochs=20)

# Evaluate the model (use your evaluate function)
evaluate(mobilenet, testloader)

# Save the model weights
torch.save(mobilenet.state_dict(), "lin_cifar_mobilenet_pretrained.pth")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 167MB/s]


Epoch [1/20], Loss: 0.9624
Epoch [2/20], Loss: 0.6805
Epoch [3/20], Loss: 0.5914
Epoch [4/20], Loss: 0.5584
Epoch [5/20], Loss: 0.4963
Epoch [6/20], Loss: 0.4621
Epoch [7/20], Loss: 0.4284
Epoch [8/20], Loss: 0.4290
Epoch [9/20], Loss: 0.3892
Epoch [10/20], Loss: 0.3639
Epoch [11/20], Loss: 0.3404
Epoch [12/20], Loss: 0.3323
Epoch [13/20], Loss: 0.3152
Epoch [14/20], Loss: 0.3039
Epoch [15/20], Loss: 0.2752
Epoch [16/20], Loss: 0.2659
Epoch [17/20], Loss: 0.2619
Epoch [18/20], Loss: 0.2541
Epoch [19/20], Loss: 0.2410
Epoch [20/20], Loss: 0.2318
Accuracy: 81.23%


In [18]:
#efficient net
from torchvision.models import efficientnet_b0

# Load pretrained EfficientNet model
efficientnet = efficientnet_b0(pretrained=True)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, 10)  # Adjust for CIFAR-10
efficientnet = efficientnet.to(device)

# Optimizer and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(efficientnet.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

# Train and evaluate
train(efficientnet, trainloader, criterion, optimizer, num_epochs=20)
evaluate(efficientnet, testloader)

#save
torch.save(efficientnet.state_dict(), "lin_cifar_efficientnet_pretrained.pth")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 120MB/s]


Epoch [1/20], Loss: 1.8316
Epoch [2/20], Loss: 1.2686
Epoch [3/20], Loss: 1.0499
Epoch [4/20], Loss: 0.9244
Epoch [5/20], Loss: 0.8310
Epoch [6/20], Loss: 0.7580
Epoch [7/20], Loss: 0.6997
Epoch [8/20], Loss: 0.6508
Epoch [9/20], Loss: 0.6126
Epoch [10/20], Loss: 0.5772
Epoch [11/20], Loss: 0.5496
Epoch [12/20], Loss: 0.5225
Epoch [13/20], Loss: 0.4945
Epoch [14/20], Loss: 0.4739
Epoch [15/20], Loss: 0.4522
Epoch [16/20], Loss: 0.4341
Epoch [17/20], Loss: 0.4163
Epoch [18/20], Loss: 0.3982
Epoch [19/20], Loss: 0.3861
Epoch [20/20], Loss: 0.3714
Accuracy: 79.59%


In [ ]:
import torch
from torchvision import models
# Define the model (same architecture as when you saved it)
num_classes = 10

#load resnet
resnet = models.resnet18(pretrained=False)
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
resnet.load_state_dict(torch.load("lin_cifar_resnet18_pretrained.pth"))
resnet = resnet.to(device)

#load densenet
densenet = models.densenet121(pretrained=False)  # Don't load the pre-trained weights initially
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)
densenet.load_state_dict(torch.load("lin_cifar_densenet_pretrained.pth"))
densenet = densenet.to(device)

#load vgg
vgg = models.vgg16(pretrained=False)
vgg.classifier[6] = nn.Linear(vgg.classifier[6].in_features, num_classes)
vgg.load_state_dict(torch.load("lin_cifar_vgg_pretrained.pth"))
vgg = vgg.to(device)

#load mobilenet
mobilenet = models.mobilenet_v2(pretrained=False)
mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, num_classes)
mobilenet.load_state_dict(torch.load("lin_cifar_mobilenet_pretrained.pth"))
mobilenet = mobilenet.to(device)

#load efficientnet
efficientnet = efficientnet_b0(pretrained=False)
efficientnet.classifier[1] = nn.Linear(efficientnet.classifier[1].in_features, num_classes)
efficientnet.load_state_dict(torch.load("lin_cifar_efficientnet_pretrained.pth"))
efficientnet = efficientnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-39-731b023d77e9>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the function

In [ ]:
def evaluate_transferability(model, patch, dataloader):
    model.eval()
    untargeted_success = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            patched_images = place_patch(images.clone(), patch)
            outputs = model(patched_images)
            _, predicted = outputs.max(1)
            untargeted_success += (predicted != labels).sum().item()
            total += labels.size(0)
    return 100 * untargeted_success / total

# Load the patch
patch = torch.load('adversarial_patch_5_5.pth')

# Evaluate transferability
resnet_rate = evaluate_transferability(model, patch, testloader)
print(f"Untargeted Attack Success Rate on ResNet: {resnet_rate:.2f}%")

densenet_rate = evaluate_transferability(densenet, patch, testloader)
print(f"Untargeted Attack Success Rate on DenseNet: {densenet_rate:.2f}%")

<ipython-input-27-ca80d6cdfda8>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  patch = torch.load('adversarial_patch_5_5.pth')


Untargeted Attack Success Rate on ResNet: 49.41%
Untargeted Attack Success Rate on DenseNet: 89.29%


In [ ]:
# Load the patch
patch = torch.load('adversarial_patch_5_5.pth')

# Evaluate transferability
resnet_rate = evaluate_transferability(model, patch, testloader)
print(f"Untargeted Attack Success Rate on ResNet: {resnet_rate:.2f}%")

densenet_rate = evaluate_transferability(densenet, patch, testloader)
print(f"Untargeted Attack Success Rate on DenseNet: {densenet_rate:.2f}%")

<ipython-input-30-d27fc4e47b23>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  patch = torch.load('adversarial_patch_5_5.pth')


Untargeted Attack Success Rate on ResNet: 48.99%
Untargeted Attack Success Rate on DenseNet: 75.71%


In [ ]:
vgg_rate = evaluate_transferability(vgg, patch, testloader)
print(f"Untargeted Attack Success Rate on VGG: {vgg_rate:.2f}%")

Untargeted Attack Success Rate on VGG: 76.72%


In [ ]:
#mobilenet
mobilenet_rate = evaluate_transferability(mobilenet, patch, testloader)
print(f"Untargeted Attack Success Rate on MobileNet: {mobilenet_rate:.2f}%")

Untargeted Attack Success Rate on MobileNet: 54.89%


In [ ]:
efficientnet_rate = evaluate_transferability(efficientnet, patch, testloader)
print(f"Untargeted Attack Success Rate on EfficientNet: {efficientnet_rate:.2f}%")

Untargeted Attack Success Rate on EfficientNet: 61.53%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import vit_b_16

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define number of classes
num_classes = 10

# Define Loss Function
criterion = nn.CrossEntropyLoss()

# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize(224),  # Vision Transformer requires 224x224 input
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize to [-1, 1]
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Load Pretrained Vision Transformer
vit = vit_b_16(pretrained=True)
vit.heads = nn.Linear(vit.heads.in_features, num_classes)  # Modify final layer for CIFAR-10
vit = vit.to(device)

# Define Optimizer and Scheduler
optimizer_vit = optim.SGD(vit.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training Function
def train(model, trainloader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")

# Evaluation Function
def evaluate(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")

# Train and Evaluate ViT
train(vit, trainloader, criterion, optimizer_vit, num_epochs=20)
evaluate(vit, testloader)

# Save the Trained Model
torch.save(vit.state_dict(), "cifar_vit_pretrained.pth")